In [10]:
import pandas as pd
import numpy as np
import oracledb
from sqlalchemy import create_engine
import os
import glob
pd.set_option('display.max_columns',100)
import time
import seaborn as sns
import matplotlib.pyplot as plt

- Perfiorming: 0 DPD
- Grace Period: 1-29 DPD
- Delinquents: Accounts with >=30 days DPD

# A. Query Data

## 1. DBA Connection

In [11]:
# credentials
usernameDBA = os.environ['DBA_USR']
passwordDBA = os.environ['DBA_PW']
dsnDBA = '10.220.50.121:1661/ANALYTIC'

In [12]:
# create connection
connectionDBA = oracledb.connect(user=usernameDBA, password=passwordDBA, dsn=dsnDBA)

In [13]:
# create engine
engineDBA = create_engine('oracle+oracledb://', creator=lambda: connectionDBA)

## 2. EDWB Connection

In [15]:
# credentials
usernameEDWB = os.environ['UID']
passwordEDWB = os.environ['EDWB_PW']
dsnEDWB = '172.16.2.28:1521/brgprod'

In [16]:
# create connection
connectionEDWB = oracledb.connect(user=usernameEDWB, password=passwordEDWB, dsn=dsnEDWB)

In [18]:
# create engine
engineEDWB = create_engine('oracle+oracledb://', creator=lambda: connectionEDWB)

## 3. Run Query

In [14]:
# create list of all dates
allDates = ['20241129','20241031','20240930','20240829','20240731','20240628','20240531',
            '20240430','20240328','20240229','20240131','20231229','20231130']
# no dpd for <202404
# ['20241129','20241031','20240930','20240829','20240731','20240628','20240531',
#             '20240430','20240328','20240229','20240131','20231229','20231130']

In [11]:
# open sql file Customer Portfolio
with open("CCPull_CP.sql", 'r') as file:
    query_CP = file.read()

# open sql file Master Lending
with open("CCPull_ML.sql", 'r') as file:
    query_ML = file.read()

# open sql file Delq_History 
with open("Delq_Hist.sql", 'r') as file:
    query_DELQ = file.read()

In [ ]:
# open sql file Master Lending grouped
with open("CCPull_ML_GPD.sql", 'r') as file:
    query_ML_SUM = file.read()

In [12]:
# RUN CP
for date in allDates:
    yearmonth = date[2:6]
    dateAlt = date[2:]
    # Query CP
    try:
        # print in case of errors
        print(f'Running CP Query {date}')

        # open and read the file as a single buffer
        sqlQuery = query_CP.format(dt=date, ym=yearmonth)

        # run query and store data in df
        print('CP run')
        with engineDBA.connect() as connection:
            CP = pd.read_sql(sqlQuery, connection) 
        print(f'CP {date} queried successfully')

        # change column names into UPPER CASE
        CP.columns = [x.upper() for x in CP.columns]
    except Exception as e:
        print(e)
        CP = pd.DataFrame()
    finally:
        CP.to_pickle(f'D:/Work/SAGA CC Bad Debt/Data/CP/CP_{date}.pkl')


Running CP Query 20241129
CP run
CP 20241129 queried successfully
Running CP Query 20241031
CP run
CP 20241031 queried successfully
Running CP Query 20240930
CP run
CP 20240930 queried successfully
Running CP Query 20240829
CP run
CP 20240829 queried successfully
Running CP Query 20240731
CP run
CP 20240731 queried successfully
Running CP Query 20240628
CP run
CP 20240628 queried successfully
Running CP Query 20240531
CP run
CP 20240531 queried successfully
Running CP Query 20240430
CP run
CP 20240430 queried successfully
Running CP Query 20240328
CP run
CP 20240328 queried successfully
Running CP Query 20240229
CP run
CP 20240229 queried successfully
Running CP Query 20240131
CP run
CP 20240131 queried successfully
Running CP Query 20231229
CP run
CP 20231229 queried successfully
Running CP Query 20231130
CP run
CP 20231130 queried successfully


In [ ]:
# # RUN CP
# try:
#     # print in case of errors
#     print(f'Running CP Query 20240628')

#     # open and read the file as a single buffer
#     sqlQuery = query_CP.format(dt='20240626', ym='2406')

#     # run query and store data in df
#     print('CP run')
#     with engineDBA.connect() as connection:
#         CP = pd.read_sql(sqlQuery, connection) 
#     print(f'CP 20240628 queried successfully')

#     # change column names into UPPER CASE
#     CP.columns = [x.upper() for x in CP.columns]
# except Exception as e:
#     print(e)
#     CP = pd.DataFrame()
# finally:
#     CP.to_pickle(f'D:/Work/SAGA CC Bad Debt/Data/CP/CP_20240628.pkl')


Running CP Query 20240628
CP run
CP 20240628 queried successfully


In [13]:
# RUN ML
for date in allDates:
    yearmonth = date[2:6]
    dateAlt = date[2:]
    # Query ML
    try:
        # print in case of errors
        print(f'Running ML Query {date}')

        # open and read the file as a single buffer
        sqlQuery = query_ML.format(dt=date, ym=yearmonth)

        # run query and store data in df
        print('ML run')
        with engineDBA.connect() as connection:
            ML = pd.read_sql(sqlQuery, connection) 
        print(f'ML {date} queried successfully')

        # change column names into UPPER CASE
        ML.columns = [x.upper() for x in ML.columns]
    except Exception as e:
        print(e)
        ML = pd.DataFrame()
    finally:
        ML.to_pickle(f'D:/Work/SAGA CC Bad Debt/Data/ML/ML_{date}.pkl')

Running ML Query 20241129
ML run
ML 20241129 queried successfully
Running ML Query 20241031
ML run
ML 20241031 queried successfully
Running ML Query 20240930
ML run
ML 20240930 queried successfully
Running ML Query 20240829
ML run
ML 20240829 queried successfully
Running ML Query 20240731
ML run
ML 20240731 queried successfully
Running ML Query 20240628
ML run
ML 20240628 queried successfully
Running ML Query 20240531
ML run
ML 20240531 queried successfully
Running ML Query 20240430
ML run
ML 20240430 queried successfully
Running ML Query 20240328
ML run
ML 20240328 queried successfully
Running ML Query 20240229
ML run
ML 20240229 queried successfully
Running ML Query 20240131
ML run
ML 20240131 queried successfully
Running ML Query 20231229
ML run
ML 20231229 queried successfully
Running ML Query 20231130
ML run
ML 20231130 queried successfully


In [16]:
# RUN ML
for date in allDates:
    yearmonth = date[2:6]
    dateAlt = date[2:]
    # Query ML
    try:
        # print in case of errors
        print(f'Running ML Sum Query {date}')

        # open and read the file as a single buffer
        sqlQuery = query_ML_SUM.format(dt=date, ym=yearmonth)

        # run query and store data in df
        print('ML Sum run')
        with engineDBA.connect() as connection:
            MLSum = pd.read_sql(sqlQuery, connection) 
        print(f'ML Sum {date} queried successfully')

        # change column names into UPPER CASE
        MLSum.columns = [x.upper() for x in MLSum.columns]
    except Exception as e:
        print(e)
        MLSum = pd.DataFrame()
    finally:
        MLSum.to_pickle(f'D:/Work/SAGA CC Bad Debt/Data/ML_Sum/MLSum_{date}.pkl')

Running ML Sum Query 20241129
ML Sum run
ML Sum 20241129 queried successfully
Running ML Sum Query 20241031
ML Sum run
ML Sum 20241031 queried successfully
Running ML Sum Query 20240930
ML Sum run
ML Sum 20240930 queried successfully
Running ML Sum Query 20240829
ML Sum run
ML Sum 20240829 queried successfully
Running ML Sum Query 20240731
ML Sum run
ML Sum 20240731 queried successfully
Running ML Sum Query 20240628
ML Sum run
ML Sum 20240628 queried successfully
Running ML Sum Query 20240531
ML Sum run
ML Sum 20240531 queried successfully
Running ML Sum Query 20240430
ML Sum run
ML Sum 20240430 queried successfully
Running ML Sum Query 20240328
ML Sum run
ML Sum 20240328 queried successfully
Running ML Sum Query 20240229
ML Sum run
ML Sum 20240229 queried successfully
Running ML Sum Query 20240131
ML Sum run
ML Sum 20240131 queried successfully
Running ML Sum Query 20231229
ML Sum run
ML Sum 20231229 queried successfully
Running ML Sum Query 20231130
ML Sum run
ML Sum 20231130 queried

In [17]:
date = '20240626'
yearmonth = date[2:6]
dateAlt = date[2:]
# Query ML
try:
    # print in case of errors
    print(f'Running ML Sum Query {date}')

    # open and read the file as a single buffer
    sqlQuery = query_ML_SUM.format(dt=date, ym=yearmonth)

    # run query and store data in df
    print('ML Sum run')
    with engineDBA.connect() as connection:
        MLSum = pd.read_sql(sqlQuery, connection) 
    print(f'ML Sum {date} queried successfully')

    # change column names into UPPER CASE
    MLSum.columns = [x.upper() for x in MLSum.columns]
except Exception as e:
    print(e)
    MLSum = pd.DataFrame()
finally:
    MLSum.to_pickle(f'D:/Work/SAGA CC Bad Debt/Data/ML_Sum/MLSum_{date}.pkl')

Running ML Sum Query 20240626
ML Sum run
ML Sum 20240626 queried successfully


In [19]:
# RUN DELQ
for date in allDates:
    yearmonth = date[2:6]
    dateAlt = date[2:]
    # query payment ratio
    try:
        # print in case of errors
        print(f'Running Delq/Payment Ratio Query {date}')

        # open and read the file as a single buffer
        sqlQuery = query_DELQ.format(dt=dateAlt, ym=yearmonth)

        # run query and store data in df
        print('Delq/Payment Ratio run')
        with engineEDWB.connect() as connection:
            PR = pd.read_sql(sqlQuery, connection) 
        print(f'Delq/Payment Ratio {date} queried successfully')

        # change column names into UPPER CASE
        PR.columns = [x.upper() for x in PR.columns]
    except Exception as e:
            print(e)
            PR = pd.DataFrame()
    finally:
        ML.to_pickle(f'D:/Work/SAGA CC Bad Debt/Data/PR/PR_{date}.pkl')

Running Delq/Payment Ratio Query 20241129
Delq/Payment Ratio run
Delq/Payment Ratio 20241129 queried successfully
Running Delq/Payment Ratio Query 20241031
Delq/Payment Ratio run
Delq/Payment Ratio 20241031 queried successfully
Running Delq/Payment Ratio Query 20240930
Delq/Payment Ratio run
Delq/Payment Ratio 20240930 queried successfully
Running Delq/Payment Ratio Query 20240829
Delq/Payment Ratio run
Delq/Payment Ratio 20240829 queried successfully
Running Delq/Payment Ratio Query 20240731
Delq/Payment Ratio run
Delq/Payment Ratio 20240731 queried successfully
Running Delq/Payment Ratio Query 20240628
Delq/Payment Ratio run
Delq/Payment Ratio 20240628 queried successfully
Running Delq/Payment Ratio Query 20240531
Delq/Payment Ratio run
Delq/Payment Ratio 20240531 queried successfully
Running Delq/Payment Ratio Query 20240430
Delq/Payment Ratio run
Delq/Payment Ratio 20240430 queried successfully
Running Delq/Payment Ratio Query 20240328
Delq/Payment Ratio run
Delq/Payment Ratio 2024